# Imports and Defaults

In [56]:
import numpy as np
import cassiopeia as cas

from cassiopeia.sim import BirthDeathFitnessSimulator
from cassiopeia.sim import Cas9LineageTracingDataSimulator

In [62]:
seed =10
rng = np.random.default_rng(seed)

# Simulate Birth Death Fitness Process

In [63]:
# instantiate BirthDeathFitnessSimulator() object and simulate tree
bd_sim = cas.sim.BirthDeathFitnessSimulator(
    birth_waiting_distribution = lambda scale: rng.exponential(scale),
    initial_birth_scale = 0.5,
    death_waiting_distribution = lambda: rng.exponential(1.5),
    mutation_distribution = lambda: 1 if rng.uniform() < 0.5 else 0,
    fitness_distribution = lambda: rng.normal(0, .5),
    fitness_base = 1.3,
    num_extant = 400,
    random_seed=17
)
ground_truth_tree = bd_sim.simulate_tree()

In [64]:
# plot tree
cas.pl.plot_plotly(ground_truth_tree)

# Simulate Cas9 Lineage & Overlay Onto Tree

In [75]:
# parameters for infintesimal generator matrix Q
num_cassettes  = 40                     # num cassettes (hyper-param)
num_states = 50                         # num states (hyper-param)

mutation_rate = [0.1] * num_cassettes   # mutation rate λ_M
deletion_rate = 9e-4                    # deletion rate λ_D
transition_prob = {i: 1/num_states for i in range(num_states)}  # P = [p_1 ... p_NumStates]
# ^ probability p_i of transitioning from unedited state to mutated state i


In [76]:
# instantiate Cas9 lineage tracing object & overlay data onto ground_truth_tree
lt_sim = cas.sim.Cas9LineageTracingDataSimulator(
    number_of_cassettes = num_cassettes,
    size_of_cassette = 1,
    mutation_rate = mutation_rate,
    state_generating_distribution = None,
    number_of_states = num_states,
    state_priors = transition_prob, # must be dict
    heritable_silencing_rate = deletion_rate,
    stochastic_silencing_rate = 0.1,
    heritable_missing_data_state = -1,
    stochastic_missing_data_state = -1,
    random_seed = seed
)
lt_sim.overlay_data(ground_truth_tree)

In [77]:
# extract character matrix
ground_truth_tree.character_matrix

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
172,0,27,34,0,0,0,0,10,0,0,...,-1,-1,0,1,2,0,24,0,0,0
225,0,27,34,0,45,0,0,41,0,0,...,0,0,0,0,22,0,-1,0,0,0
238,0,27,0,0,0,12,0,0,38,0,...,16,-1,-1,0,0,0,0,0,0,0
295,3,27,34,0,-1,0,0,0,12,0,...,0,0,11,0,2,0,25,0,0,0
297,0,27,34,0,0,0,0,0,0,24,...,0,0,0,0,0,0,0,-1,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,0,-1,34,48,0,0,-1,0,0,17,...,0,0,0,11,0,0,42,0,4,0
1150,0,27,34,0,0,0,0,-1,6,0,...,0,0,0,0,2,0,0,0,-1,0
1151,0,27,34,0,-1,0,0,10,6,0,...,0,0,0,0,2,0,0,0,0,0
1152,0,-1,34,48,0,-1,0,0,0,0,...,-1,0,27,11,0,0,-1,0,0,0
